### LangChain ?

- LangChain은 언어 모델에 의해 작동하는 애플리케이션을 개발하기 위한 프레임워크이다.

**TLDR**

LangChain은 아래의 2가지 방법를 통해 AI 모델을 작동하거나 빌드하는 복잡한 과정을 더 쉽게 할 수 있도록 한다.

1. Integration - 파일, 다른 애플리케이션이나 API 데이터와 같은 외부 데이터를 쉽게 가져올 수 있다.

2. Agency - LLMs가 주어진 환경과 상호 작용하도록 하여 다음 수행할 작업을 결정하는데 도움을 준다.

### Why LangChain ?

1. Components - LangChain을 사용하면 언어 모델에 작동하기에 필요한 구성요소들을 쉽게 다룰 수 있다.

2. Customized Chains - LangChain은 커스터마이징이 가능한 chains라는 컴포넌트를 지원한다.

3. Speed - LangChain팀은 매우 빠른 속도로 개발하기 때문에 LLMs의 최신 기능들을 사용할 수 있다.

4. Community - 디스코드와 같은 커뮤니티가 활성화 되어 있다.

In [1]:
openai_api_key = 'YourAPIKey'

# LangChain Components

## Schema - Nuts and Bolts of working with LLMs

### Text

The natural language way to interact with LLMs

In [2]:
my_text = "What day comes after Friday?"

### Chat Messages

- **System** - AI에게 무엇을 해야하는지 알려주는 중요한 배경

- **Human** - 사용자의 메세지

- **AI** - AI의 메세지

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

In [4]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)

AIMessage(content='How about a fresh caprese salad with ripe tomatoes, mozzarella cheese, and basil?')

In [5]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to Nice, France"),
        HumanMessage(content="What else should I do when I'm there?"),
    ]
)

AIMessage(content='You should explore the charming old town and indulge in the delicious local cuisine.')

### Documents

- 메타데이터와 텍스트를 가지고 있는 객체

In [6]:
from langchain.schema import Document

Document(page_content="This is my document. It is full of text that I've gathered from other places",
         metadata={
             'my_document_id' : 123123,
             'my_document_source' : 'The LangChain Papers',
             'my_document_create_time' : 1680013019
         })

Document(page_content="This is my document. It is full of text that I've gathered from other places", metadata={'my_document_id': 123123, 'my_document_source': 'The LangChain Papers', 'my_document_create_time': 1680013019})

## Models

#### Language Model
[OpenAI models](https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo)

In [23]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=openai_api_key)

In [24]:
llm("What day comes after Friday?")

'\n\nSaturday'

#### Language Model

In [26]:
from langchain_community.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key)

In [27]:
chat(
    [
        SystemMessage(content='You are an unhelpful AI bot that makes a joke at whatever the user says'),
        HumanMessage(content="I would like to go to New York. how should I do this?")
    ]
)

AIMessage(content='Why did the scarecrow win an award? Because he was outstanding in his field! But seriously, to get to New York, you could try walking on water, but it might be easier to just take a plane or a teleportation device if you have one handy. Good luck!')

#### [Function Calling](https://openai.com/blog/function-calling-and-other-api-updates)

GPT는 함수를 실행하지 않고 함수를 실행할 때 필요한 인자를 알려줄 수 있다.  
실제 실행은 사용자가 직접 하고, 결과를 다시 GPT에게 제공하여 그 결과를 활용할 수 있다.

In [44]:
import json

In [45]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key)

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


output = chat(messages=[
            SystemMessage(content="You are an helpful AI bot"),
            HumanMessage(content="What's the weather like in Boston right now?")
            ],
            functions=[
                  {"name" : "get_current_weather",
                   "description" : "Get the current weather in a given location",
                   "parameters": {
                       "type": "object",
                       "properties": {
                           "location": {
                               "type" : "string",
                               "description" : "The city and state, e.g, San Francisco, CA"
                           },
                           "unit" : {
                               "type" : "string",
                               "enum" : ["celsius", "fahrenheit"]
                           }
                       },
                       "required" : ["location"]
                       }
                   }
            ]
            )

output

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "location": "Boston, MA"\n}', 'name': 'get_current_weather'}})

#### Text Embedding Model

텍스트를 벡터로 변환

In [31]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.6/955.6 kB 15.8 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 8.7 MB/s eta 0:00:00


In [28]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [29]:
text = "Hi! It's time for the beach"

In [32]:
text_embedding = embeddings.embed_query(text)
print(f"Embedding is length {len(text_embedding)}")
print(f"Here's a sample: {text_embedding[:5]}...")

Embedding is length 1536
Here's a sample: [-0.00019600906371495047, -0.0031846734422911363, -0.0007734206914647714, -0.019472001962491232, -0.015092319017854244]...


## Prompts - Text generally used as instructions to your model


#### Prompt

모델에 전달할 내용

In [41]:
from langchain.llms import OpenAI

llm = OpenAI( openai_api_key=openai_api_key)

In [42]:
prompt = """
Today is Monday, tomorrow is Wednesday.

Waht is wrong with that statement?
"""

print(llm(prompt))


The statement is incorrect because it skips over Tuesday and jumps directly from Monday to Wednesday.


#### Prompt Templates

In [43]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(openai_api_key=openai_api_key)
template = """
I really want to travel to {localtion}. what should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=['localtion'],
    template=template
)

final_prompt = prompt.format(localtion='Rome')

print(f"Final Prompt: {final_prompt}")
print('-------------------')
print(f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
I really want to travel to Rome. what should I do there?

Respond in one short sentence

-------------------
LLM Output: 
Visit famous historical sites, try authentic Italian food, and immerse yourself in the vibrant culture.


#### [Example Selectors](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/)

상황에 맞는 정보를 프롬프트에 동적으로 배치할 수 있는 작업을 방법이다.

In [48]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [49]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=openai_api_key)

example_prompt = PromptTemplate(
    input_variables=['input', 'output'],
    template="Example Input: {input}\nExample Output: {output}"
)


examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]


In [50]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 예제 리스트
    examples,
    # 벡터화를 하기 위한 모델
    OpenAIEmbeddings(openai_api_key=openai_api_key),
    # 벡터를 저장하고 유사도 검색을 할 수 있도록 하는 벡터 저장소
    FAISS,
    # 생성할 예제 수
    k=2
)

In [51]:
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    
    input_variables=["noun"]
)

In [52]:
my_noun = "plant"

print(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: tree
Example Output: ground

Example Input: bird
Example Output: nest

Input: plant
Output:


In [53]:
llm(similar_prompt.format(noun=my_noun))

' ground'

#### Output Parsers

구조화된 출력을 원할 때 유용한 방법 - [docs](https://python.langchain.com/docs/modules/model_io/output_parsers)

1. Format Instructions - 원하는 출력을 LLM에 알려주는 자동 생성 프롬프트

2. Parser - 모델의 텍스트 출력을 원하는 구조로 추출하는 방법

In [54]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [55]:
llm = OpenAI(openai_api_key=openai_api_key)

In [56]:
# 응답 구성
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# 출력 구성
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [58]:
format_instructions = output_parser.get_format_instructions()
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [59]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT %
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to califonya!")
print(promptValue)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT %
welcom to califonya!

YOUR RESPONSE:



In [60]:
llm_output = llm(promptValue)
print(llm_output)

```json
{
	"bad_string": "welcom to califonya!"
	"good_string": "Welcome to California!"
}
```


## Index

#### Document Loaders
: PDF, HTML, Text, Markdown, Json, CSV 파일 외에도 다양한 종류의 로컬 파일들과 클라우드 드라이브의 파일들을 읽을 수 있는[ Document loader]((https://python.langchain.com/docs/modules/data_connection/document_loaders.html))들이 있다.

In [64]:
!pip install beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 1.6 MB/s eta 0:00:002.0 MB/s eta 0:00:01


In [65]:
from langchain.document_loaders import HNLoader

**HackerNews**

In [66]:
loader = HNLoader("https://news.ycombinator.com/item?id=34422627")

In [67]:
data = loader.load()

In [69]:
print(f"Found {len(data)} comments")
print(f"Here's a sample: \n\n{''.join([x.page_content[:150] for x in data[:2]])}")

Found 76 comments
Here's a sample: 

Ozzie_osman on Jan 18, 2023  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are veOzzie_osman on Jan 18, 2023  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_ind


**URLs and webpages**

In [72]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 808.1 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 1.5 MB/s eta 0:00:001.6 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 1.2 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 1.4 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 1.6 MB/s eta 0:00:001.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.0 MB/s eta 0:00:001.1 MB/s eta 0:00:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=16050f587184969a841dd95615978c0a8b5586465d2083602e84030af85d10af
  Stored in directory: /Users/khs/Library/Caches/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa

In [74]:
from langchain.document_loaders import UnstructuredURLLoader

urls = [
    "http://www.paulgraham.com/"
]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()
data[0].page_content

'New: Superlinear Returns | How to Do Great Work Want to start a startup? Get funded by Y Combinator . © mmxxiii pg'

#### Text Splitters
: 긴 텍스트들을 편리하게 제어할 수 있는 기능

In [75]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [78]:
with open("./long_doc.txt") as f:
    data = f.read()
    
print(f"You have {len([data])} document")

You have 1 document


In [79]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 150,
    chunk_overlap = 20,
)

texts = text_splitter.create_documents([data])

In [80]:
print(f"You Have {len(texts)} documents.")

You Have 14 documents.


In [81]:
print("Preview:")
print(texts[0].page_content, '\n')
print(texts[1].page_content)

Preview:
this is very long document.
this is very long document.
this is very long document.
this is very long document.this is very long document. 

this is very long document.this is very long document.
this is very long document.this is very long document.this is very long document.


### Retrievers
: LLMs와 문서들을 연결할 수 있는 기능  
: VectoreStoreRetriever가 가장 많이 사용되고 있다.

In [89]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loaders = TextLoader('./long_doc.txt')
documents = loaders.load()

In [90]:
# text splitter 준비
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

# doc 분할
texts = text_splitter.split_documents(documents)

# 임베딩 모델 준비
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# 텍스트 임베딩
db = FAISS.from_documents(texts, embeddings)

In [91]:
# retriever 초기화
retriever = db.as_retriever()

In [92]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x291898400>)

In [93]:
docs = retriever.get_relevant_documents("I have a long stick")

In [94]:
print("\n\n".join([x.page_content[:50] for x in docs[:2]]))

is very long document.

is very long document.


#### VectorStores
: 벡터를 저장하는 데이터베이스  
: **pinecone**, **weaviate**가 가장 유명하다.[참고](https://github.com/openai/chatgpt-retrieval-plugin#choosing-a-vector-database)  
: **chroma**, **faiss**는 로컬에서 개발하기 편리하다.

개념적으로 임베딩벡터와 메타데이터로 이루어진 테이블이라고 생각할 수 있다.

|Embedding|Metadata|
|:--:|:--:|
|[-0.112..., 0.4432...]|{'date': '1/2/3'...}|


In [95]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('./long_doc.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)

texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [96]:
print (f"You have {len(texts)} documents")

You have 22 documents


In [97]:
embedding_list = embeddings.embed_documents([text.page_content for text in texts])

In [98]:
print (f"You have {len(embedding_list)} embeddings")
print (f"Here's a sample of one: {embedding_list[0][:3]}...")

You have 22 embeddings
Here's a sample of one: [-0.005622338760664389, 0.009526740406156697, -0.003897611605683349]...


## Memory
: LLMs가 정보를 기억할 수 있도록 돕는 기능 [Memory](https://python.langchain.com/docs/modules/memory/)

In [101]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

history = ChatMessageHistory()

history.add_ai_message("hi!")

history.add_user_message("what is the capital of france?")

In [102]:
history.messages

[AIMessage(content='hi!'),
 HumanMessage(content='what is the capital of france?')]

In [103]:
ai_response = chat(history.messages)
ai_response

AIMessage(content='The capital of France is Paris.')

In [104]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='hi!'),
 HumanMessage(content='what is the capital of france?'),
 AIMessage(content='The capital of France is Paris.')]

## [Chains](https://python.langchain.com/docs/modules/chains)

: 다양한 LLM을 호출하고 작업을 결합하는 기능


### 1. Simple Sequential Chains

In [105]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

llm = OpenAI(temperature=1, openai_api_key=openai_api_key)

In [107]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [109]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [110]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

In [111]:
review = overall_chain.run("Rome")



> Entering new SimpleSequentialChain chain...
For Rome, a classic dish that comes to mind is Spaghetti alla Carbonara. This pasta dish is made with spaghetti, pancetta or guanciale (cured pork cheek), eggs, pecorino romano cheese, and black pepper. The eggs and cheese create a creamy and rich sauce that pairs perfectly with the salty pancetta. Spaghetti alla Carbonara is a staple in Roman cuisine and is loved by both locals and tourists. It's a simple yet delicious dish that represents the flavors and traditions of Rome. 
To make Spaghetti alla Carbonara at home, start by bringing a pot of salted water to a boil and cooking spaghetti according to package instructions. While the pasta is cooking, heat a large skillet over medium high heat and add in diced pancetta or guanciale. Cook until crispy, then remove from the pan and set aside. In a separate bowl, whisk together eggs, grated pecorino romano cheese, and black pepper. Once the pasta is al dente, drain and add it directly to the 

### 2. Summarization Chain

In [112]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('./news.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

texts = text_splitter.split_documents(documents)

chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"[디지털투데이 AI리포터] 오픈AI의 인공지능(AI) 챗봇인 챗GPT가 사용자의 로그인 정보와 개인 정보를 유출했다는 주장이 제기됐다고 30일 IT매체 아스테크니카가 전했다. 

정보 유출 피해를 입었다고 주장하는 제보자가 제출한 스크린샷에는 약국 처방약 포털의 직원이 사용하는 지원 시스템에 연결된 제보자 이름과 비밀번호가 포함돼 있었다. 이 외에도 앱의 이름과 문제가 발생한 스토어 번호 등이 포함돼 있는 것이 나타났다. 

이에 대해 제보자는 "이건 정말 말도 안 되고 끔찍한 일"이라며 비난했다. 제보자는 "애초에 챗GPT의 시스템이 얼마나 엉망으로 만들어 진 건가"라며 우려를 제기했다.

이와 같은 사례는 AI 서비스에서 개인 정보를 최대한 배제해야 한다는 점을 상기시킨다. 앞서 2023년 3월에는 챗GPT의 버그로 인해 사이트에서 한 활성 사용자의 채팅 기록 제목이 관련 없는 사용자에게 표시되는 현상이 발생한 바 있다. 이에 오픈AI는 챗GPT를 오프라인으로 전환했다.

이러한 유형의 시스템 오류는 종종 발생할 수 있다. 정확한 원인은 인시던트마다 다르지만, 프론트엔드 디바이스와 백엔드 디바이스 사이에 있는 미들박스 디바이스와 관련된 경우가 많은 것으로 알려졌다."


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"미들박스는 성능 향상을 위해 최근에 로그인한 사용자의 자격 증명을 비롯한 특정 데이터를 캐시하는 기능이다. 이때 불일치가 발생하면 한 계정의 정보를 다른 계정에 매핑하는 등의 문제가 발생할 수 있다. 

해당 논란과 관련해 오픈AI 담당자는 이번 사건에 대해 아직 

' According to AI reporters, there have been claims of a GPT chatbot leaking user login and personal information, leading to backlash. This serves as a reminder for AI services to minimize personal information and cache data responsibly. OpenAI has previously encountered similar bugs and is currently investigating this issue.'

### Agents

: 에이전트는 체인이 사용할 도구(Tools)들을 지원하며 LLMs들이 원하는 작업을 수행하도록 절차를 정의할 수 있다.



**Tools**  

Agents들이 LLMs와 쉽게 상호 작용할 수 있도록 추상화한 기능. [Tools](https://python.langchain.com/docs/integrations/tools/) / [CustomAgent](https://python.langchain.com/docs/modules/agents/how_to/custom_agent)



In [114]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
import json
import os

llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [117]:
serpapi_api_key=os.getenv("SERP_API_KEY", "YourAPIKey")

In [118]:
toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=serpapi_api_key)

In [119]:
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

In [120]:
response = agent({"input":"빅뱅의 첫번째 앨범은 무엇인가요?"})



> Entering new AgentExecutor chain...
 I should search for the first album of Big Bang
Action: Search
Action Input: Big Bang first album
Observation: ['Big Bang is V.I.P is the second single album by the South Korean boy band Big Bang, released by YG Entertainment on September 28, 2006. It spawned the single "La La La".', 'Big Bang Is V.I.P type: Album by BIGBANG.', 'Big Bang Is V.I.P artist: BIGBANG.', 'Big Bang Vol. 1 – Since 2007 is the debut studio album by South Korean boy band BigBang, released on December 22, 2006 through YG Entertainment.', 'The release of their debut album, BigBang Vol. 1 – Since 2007 (2006) followed. Debuting at no. 3, the album went on to sell over 110,000 copies.', 'BIGBANG* – First Single Album ; Have:20 ; Want:10 ; Avg Rating:4 / 5 ; Ratings:1 ; Last Sold:Never ...', 'BIGBANG - BIGBANG [First Single Album] 1st Single Album CD+etc+Tracking Number K-POP SEALED - Amazon.com Music.', '1. Big Bang (8) - Bigbang Vol.1 album cover. More images. Genre: Electron

In [125]:
from langchain.load.dump import dumps

print(dumps(response["intermediate_steps"], pretty=True))

[
  [
    {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "schema",
        "agent",
        "AgentAction"
      ],
      "kwargs": {
        "tool": "Search",
        "tool_input": "Big Bang first album",
        "log": " I should search for the first album of Big Bang\nAction: Search\nAction Input: Big Bang first album"
      }
    },
    "['Big Bang is V.I.P is the second single album by the South Korean boy band Big Bang, released by YG Entertainment on September 28, 2006. It spawned the single \"La La La\".', 'Big Bang Is V.I.P type: Album by BIGBANG.', 'Big Bang Is V.I.P artist: BIGBANG.', 'Big Bang Vol. 1 \u2013 Since 2007 is the debut studio album by South Korean boy band BigBang, released on December 22, 2006 through YG Entertainment.', 'The release of their debut album, BigBang Vol. 1 \u2013 Since 2007 (2006) followed. Debuting at no. 3, the album went on to sell over 110,000 copies.', 'BIGBANG* \u2013 First Single Album ; Have:20 ; Wa